In [2]:
import tweepy
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

from config import (consumer_key, 
                    consumer_secret, 
                    access_token, 
                    access_token_secret)

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

In [3]:
targets = ["@BBCNews", "@CBSNews", "@CNN", "@FoxNews", "@nytimes"]


In [ ]:
For user in targets:
    index = -1
    index += 1
    counter = 0
    oldest_tweet = 0
    for x in range(5):
        public_tweets = api.user_timeline(user, max_id=oldest_tweet)
        
        for tweet in public_tweets:
            results = analyzer.polarity_scores(tweet["text"])
            compound = results["compound"]
            pos = results["pos"]
            neu = results["neu"]
            neg = results["neg"]
            tweets_ago = counter
            
            oldest_tweet = tweet["id"]-1
            
            counter += 1